### Where are my trees?

This notebook runs through the method to identify fiducial markers (akin to QR codes) in laser scans. Currently only the AruCo `aruco_mip_16h3` dictionary of codes is compatible.

In [ ]:
%pylab
%matplotlib inline
import os
import glob
import pandas as pd

import qrdar

Make a directory called <code><span style="background-color:lightgrey">fiducial markers</span></code> navigate to it 

In [ ]:
%%bash
cd /some/directory/somewhere
mkdir fiducial_markers
mkdir clusters

In [ ]:
os.chdir('/data/eos/TLS/australia/AEP-33/2018-08-24.001.riproject/extraction2delete/fiducial_markers/')

### Load point cloud
Load point cloud and filter leaving only bright points e.g. for the <i>REIGL VZ-400</i>, points where reflectance > 0 db should do. There is a function to load points from a <code><span style="background-color:lightgrey">.pcd</span></code> (the output of <code><span style="background-color:lightgrey">mon2pcd</span></code> if using a RIEGL workflow) but any pd.DataFrame with <code><span style="background-color:lightgrey">['x', 'y', 'z']</span></code> columns will do.

In [ ]:
pc = qrdar.read('../aep33_mon.pcd')

## If area is large 
For example if you have a large forest plot, where the data is tiled

### Search for potential stickers
This step groks pc looking for clusters of points that could be stickers (or something else e.g. co-registration targets, reflective strips on bags etc.). These are then filtered in using <code><span style="background-color:lightgrey">filterBySize</span></code>

In [ ]:
pc = qrdar.search4stickers.find(pc)

# as this step can take a while it is a good idea to save the output to read in at a later date.
qrdar.io.write_ply('potential_points.ply', pc)

In [ ]:
# read in previously identified points
pc = qrdar.io.read_ply('potential_points.ply')

In [ ]:
pc = qrdar.search4stickers.filterBySize(pc, verbose=True)

### Group stickers into potential tree codes
Stickers are grouped into potential tree codes 

This step is not neccessary as it happens in `readMarkersFromTiles` but can be used for visualosation e.g. in CloudCompare.

In [ ]:
pc = qrdar.readMarker.locateTargets(pc, verbose=True)
qrdar.io.write_ply('target_centres.ply', pc.groupby('target_labels_').mean())

### Read codes
If codes are spread over a large dataset it is common for that dataset to be tiled therefore you would need a tile index and to run using <code><span style="background-color:lightgrey">readMarkersFromTiles()</span></code> method.

This requires a tile index including the tile x and y centre as well as tile number. 

In [ ]:
tile_index = pd.read_csv('../tile_index.dat', names=['tile', 'x', 'y'], sep=' ')
marker_df = qrdar.readMarker.readMarkersFromTiles(pc, tile_index, '../rxp2pcd_i/aep33_{}.pcd', verbose=True)

### Extract features
Once codes have been identified the features they are attachted to can be extracted. 

In [ ]:
qrdar.extractFeatures(marker_df, tile_index, 
                      '../downsample_p/aep33_{}.downsample.pcd', 
                      '../clusters/', verbose=True)